In [1]:
import pandas as pd
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime,timedelta
from itertools import product
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [2]:
keys = pd.read_excel(r'key_match_2019_16-18_data.xlsx',sheetname='final_keys')['keys'].tolist()

df_16_18 = pd.read_pickle(r'df_16_18.pkl')
df_2019 = pd.read_pickle(r'df_2019.pkl')

df_16_18.columns = ['Distributor', 'STRIPES_Code', 'month-year', 'Date', 'year','SIV', 'timestamp', 'key', 'if_exist']
df_2019.columns = ['Distributor', 'Material', 'timestamp', 'YEAR', 'EMAPS_Forecast', 'SIV',\
       'is_demand', 'key', 'if_in_demand']

In [3]:
# fixed params
base_time_frame = pd.date_range(start="2016-01-01",end="2020-02-01",freq="MS")
train_data_max_date = datetime.strptime('20190701',"%Y%m%d")
forecast_range = list(filter(lambda x: x > train_data_max_date,base_time_frame))
res = list()

In [4]:
res = list()
for key in keys:
    ser_16_18 = df_16_18.query('key == @key')[['timestamp','SIV']].set_index('timestamp')
    ser_2019 = df_2019.query('key == @key')[['timestamp','SIV']].set_index('timestamp')
    ser = ser_16_18.append(ser_2019).sort_index()
    #ser = ser[ser<=ser.quantile(0.99)]
    ser = ser[ser < ser.quantile(0.75)+ (ser.quantile(0.75)-ser.quantile(0.25))*2]
    tm_df = pd.DataFrame(index=base_time_frame)
    tm_df = pd.concat([tm_df,ser],axis=1)
    sample_data = tm_df.interpolate(method='time')
    tm_df.fillna(0,inplace=True)
    print(key)
    
    for tm in forecast_range:
        train_data = sample_data.loc[:tm][:-1]
        demand_act = tm_df.loc[tm].values[0]
        naive_fcst = train_data.iloc[-1].values[0]
        avg_fcst =  train_data.mean()[0]
        sma2 = train_data.rolling(2).mean().iloc[-1][0]
        sma5 = train_data.rolling(5).mean().iloc[-1][0]
        fit2 = SimpleExpSmoothing(np.asarray(train_data)).fit(smoothing_level=0.6,optimized=False)
        ses = fit2.forecast(1)[0]
        fit1 = Holt(np.asarray(train_data)).fit(smoothing_level = 0.3,smoothing_slope = 0.1)
        holt = fit1.forecast(1)[0]
        hwm_model = ExponentialSmoothing(np.asarray(train_data) ,seasonal_periods=7 ,trend='add', seasonal
                ='add',).fit()
        hwm = hwm_model.forecast(1)[0]
        #print(tm.date())
        fcst = 0
        stderr = 0
        up_conf = 0
        low_conf = 0
        try:
            model = ARIMA(train_data, order=(2,1,1))
            model_fit = model.fit(transparams=False,disp=-1,trend='c',maxiter = 5000)#start_params =5
            #print(model_fit.forecast())
            fcst,stderr,conf = model_fit.forecast()
            fcst = 0 if np.isnan(round(fcst[0])) else round(fcst[0])
            stderr = 0 if np.isnan(round(stderr[0])) else round(stderr[0])
            low_conf = 0 if np.isnan(round(conf[0][0])) else round(conf[0][0])
            up_conf = 0 if np.isnan(round(conf[0][1])) else round(conf[0][1])
            #print(fcst,stderr)
        except Exception as e:
            print(e)
            fcst = 0
            fcst = 0
            stderr = 0
            up_conf = 0
            low_conf = 0
        
        res.append([key,tm,demand_act,fcst,stderr,low_conf,up_conf,naive_fcst,\
                    avg_fcst,sma2,sma5,ses,holt,hwm]) #tm.strftime("%Y-%m-%d")
res_df = pd.DataFrame(res,columns=['key','timestamp','actual','forecast','stdErr','lowPrInt','upPrInt','naive',\
                                  'avg_fcst','sma2','sma5','ses','holt','hwm'])

Super Transports P Ltd-139542
Super Transports P Ltd-139530
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
Super Transports P Ltd-122310
Super Transports P Ltd-139546
SVD did not converge
Super Transports P Ltd-139306
Super Transports P Ltd-139434
Super Transports P Ltd-139495
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
Super Transports P Ltd-122330
Super Transports P Ltd-139461
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
Super Transports P Ltd-13

In [7]:
res_df_2 = pd.merge(res_df,df_2019[['key','EMAPS_Forecast','timestamp']],how='left',on=['key','timestamp'])
res_df_2['fcst2'] = res_df_2['forecast']
res_df_2.loc[(res_df_2.fcst2==0),'fcst2'] = res_df_2.naive

res_df_3 = res_df_2.query('0<= forecast <= 6267')

res_df_3 = res_df_3.loc[res_df_3.actual != 0]

mae1 = (res_df_3.actual-res_df_3.forecast).abs().mean()
mae2 = (res_df_3.actual-res_df_3.EMAPS_Forecast).abs().mean()
mae3 = (res_df_3.actual-res_df_3.naive).abs().mean()
mae4 = (res_df_3.actual-res_df_3.fcst2).abs().mean()

mae5 = (res_df_3.actual-res_df_3.avg_fcst).abs().mean()
mae6 = (res_df_3.actual-res_df_3.sma2).abs().mean()
mae7 = (res_df_3.actual-res_df_3.sma5).abs().mean()
mae8 = (res_df_3.actual-res_df_3.ses).abs().mean()
mae9 = (res_df_3.actual-res_df_3.holt).abs().mean()
mae10 = (res_df_3.actual-res_df_3.hwm).abs().mean()

print(mae1,mae2,mae3,mae4,mae5,mae6,mae7,mae8,mae9,mae10)

2203.525506828528 1857.4497738998484 1983.0589683801356 1793.8420879602247 1572.8081319160462 1646.4687011006122 1491.2038869072012 1155.38173712664 1078.4755540510987 1114.5804065690204


In [ ]:
# still to try SARIMAX but need hyper parameter tuning for the models will take time 

In [9]:
res_df_3.to_excel('model_results_2.xlsx',index=False)

In [10]:
 train_data.mean()[0]

1013.4661184252885

In [8]:
mae10

1114.5804065690204

In [10]:
res_df_3.columns

Index(['key', 'timestamp', 'actual', 'forecast', 'stdErr', 'lowPrInt',
       'upPrInt', 'naive', 'avg_fcst', 'sma2', 'sma5', 'ses', 'holt', 'hwm',
       'EMAPS_Forecast', 'fcst2'],
      dtype='object')

In [11]:
res_df_3.actual.sum()

1849655.6649999998

In [14]:
mae1 = 1-(res_df_3.actual-res_df_3.forecast).abs().sum()/res_df_3.actual.sum()
mae2 = 1-(res_df_3.actual-res_df_3.EMAPS_Forecast).abs().sum()/res_df_3.actual.sum()
mae3 = 1-(res_df_3.actual-res_df_3.naive).abs().sum()/res_df_3.actual.sum()
mae4 = 1-(res_df_3.actual-res_df_3.fcst2).abs().sum()/res_df_3.actual.sum()

mae5 = 1-(res_df_3.actual-res_df_3.avg_fcst).abs().sum()/res_df_3.actual.sum()
mae6 = 1-(res_df_3.actual-res_df_3.sma2).abs().sum()/res_df_3.actual.sum()
mae7 = 1-(res_df_3.actual-res_df_3.sma5).abs().sum()/res_df_3.actual.sum()
mae8 = 1-(res_df_3.actual-res_df_3.ses).abs().sum()/res_df_3.actual.sum()
mae9 = 1-(res_df_3.actual-res_df_3.holt).abs().sum()/res_df_3.actual.sum()
mae10 = 1-(res_df_3.actual-res_df_3.hwm).abs().sum()/res_df_3.actual.sum()

print(mae1,mae2,mae3,mae4,mae5,mae6,mae7,mae8,mae9,mae10)

0.2149223574540291 0.338223095161877 0.2934707335581245 0.36088540243743783 0.4396359395181402 0.4133919655660322 0.4687101063906153 0.7732531635411338 0.7883462130122749 0.781260536628284


In [15]:
print(mae1*100,mae2*100,mae3*100,mae4*100,mae5*100,mae6*100,mae7*100,mae8*100,mae9*100,mae10*100)

21.49223574540291 33.8223095161877 29.347073355812448 36.088540243743786 43.96359395181402 41.33919655660322 46.87101063906153 77.32531635411338 78.83462130122749 78.1260536628284
